In [ ]:
!pip install -U pip
!pip install ktrain

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import os

import nltk
nltk.download('punkt')
from nltk import word_tokenize

import string
import re

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
train_df = pd.read_csv('/content/Train.csv')
test_df = pd.read_csv('/content/test.csv')

In [ ]:
train_df

,id,content,category
0,SW0,SERIKALI imesema haitakuwa tayari kuona amani...,Kitaifa
1,SW1,"Mkuu wa Mkoa wa Tabora, Aggrey Mwanri amesiti...",Biashara
2,SW10,SERIKALI imetoa miezi sita kwa taasisi zote z...,Kitaifa
3,SW100,KAMPUNI ya mchezo wa kubahatisha ya M-bet ime...,michezo
4,SW1000,WATANZANIA wamekumbushwa kusherehekea sikukuu...,Kitaifa
...,...,...,...
5146,SW993,RAIS John Magufuli ameendelea kung’ara katika...,Kitaifa
5147,SW994,KAMPENI inayohimiza watafi ti kuandika upya h...,Kitaifa
5148,SW996,"MATUKIO mapya ya malaria kwa kila watu 1,000 ...",Kitaifa
5149,SW997,IMEELEZWA kuwa hakuna sheria yoyote inayokata...,Kitaifa


In [ ]:
train_df.category.value_counts()

Kitaifa      2000
michezo      1720
Biashara     1360
Kimataifa      54
Burudani       17
Name: category, dtype: int64

In [ ]:
# encoded_df = pd.get_dummies(train_df['category'])

# train_df = pd.concat([train_df, encoded_df], axis=1)
# train_df = train_df.drop(columns='category')

In [ ]:
def clean_text(text):
  text = text.lower()
  text = re.sub('â€˜', '', text)
  text = re.sub('[^\w\s]', '', text)
  text = re.sub('\d+', '', text)
  text = word_tokenize(text)
  return ' '.join(text)

clean_text('SERIKALI imesema haitakuwa tayari kuona amani.')

'serikali imesema haitakuwa tayari kuona amani'

In [ ]:
train_df['content'] = train_df.content.apply(clean_text)
train_df

,id,content,category
0,SW0,serikali imesema haitakuwa tayari kuona amani ...,Kitaifa
1,SW1,mkuu wa mkoa wa tabora aggrey mwanri amesitish...,Biashara
2,SW10,serikali imetoa miezi sita kwa taasisi zote za...,Kitaifa
3,SW100,kampuni ya mchezo wa kubahatisha ya mbet imein...,michezo
4,SW1000,watanzania wamekumbushwa kusherehekea sikukuu ...,Kitaifa
...,...,...,...
5146,SW993,rais john magufuli ameendelea kungara katika s...,Kitaifa
5147,SW994,kampeni inayohimiza watafi ti kuandika upya hi...,Kitaifa
5148,SW996,matukio mapya ya malaria kwa kila watu katika ...,Kitaifa
5149,SW997,imeelezwa kuwa hakuna sheria yoyote inayokataz...,Kitaifa


In [ ]:
# from sklearn.model_selection import train_test_split

# X = train_df['content']
# y = train_df[['Biashara', 'Burudani', 'Kimataifa', 'Kitaifa', 'michezo']]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

In [ ]:
df = train_df[['content', 'category']]
df_train = df.sample(frac=0.85, random_state=2020)
df_test = df.drop(df_train.index)
len(df_train), len(df_test)

(4378, 773)

In [ ]:
import ktrain
from ktrain import text

MODEL_NAME = 'bert-base-multilingual-uncased'

MAXLEN  = 128
batch_size = 32
learning_rate = 5e-5
epochs = 3

In [ ]:
transformer = text.Transformer(MODEL_NAME, maxlen=MAXLEN)
trn = transformer.preprocess_train(df_train.content.values, df_train.category.values)
val = transformer.preprocess_test(df_test.content.values, df_test.category.values)
model = transformer.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=batch_size)
learner.fit(learning_rate, epochs)

preprocessing train...
language: sw
train sequence lengths:
	mean : 309
	95percentile : 663
	99percentile : 1052


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: sw
test sequence lengths:
	mean : 304
	95percentile : 614
	99percentile : 1105


Epoch 1/3
137/137 [==============================] - 4326s 31s/step - loss: 0.7069 - accuracy: 0.7060 - val_loss: 0.6377 - val_accuracy: 0.7464
Epoch 2/3
137/137 [==============================] - 4352s 32s/step - loss: 0.4332 - accuracy: 0.8392 - val_loss: 0.3901 - val_accuracy: 0.8499
Epoch 3/3
137/137 [==============================] - 4326s 32s/step - loss: 0.3462 - accuracy: 0.8687 - val_loss: 0.4129 - val_accuracy: 0.8305
